# Correlation Analysis at Goal-Target Level

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Import libraries and read in data

In [2]:
import pandas as pd
import numpy as np

In [8]:
#Read in data
Indonesia=pd.read_csv('https://github.com/PeishanLi/G5055_Practicum_Project2/raw/main/Data/processedIndo.csv')
Guatemala=pd.read_csv('https://github.com/PeishanLi/G5055_Practicum_Project2/raw/main/Data/processedGuate.csv')
Ivariables=pd.read_csv('https://github.com/PeishanLi/G5055_Practicum_Project2/raw/main/Data/variables_picked/Indo_GTIM.csv')
Gvariables=pd.read_csv('https://github.com/PeishanLi/G5055_Practicum_Project2/raw/main/Data/variables_picked/Gua_GTIM.csv')

#A little preprocessing of data
Indonesia=Indonesia.drop('Unnamed: 0', axis=1)
Indonesia['UniqueIDNew']=[x.replace(" ","") for x in Indonesia['UniqueID']]
Guatemala=Guatemala.drop('Unnamed: 0', axis=1)
Guatemala['UniqueIDNew']=[x.replace(" ","") for x in Guatemala['UniqueID']]
Ivariables['UniqueIDNew']=[x.replace(" ","") for x in Ivariables['UniqueID']]
Gvariables['UniqueIDNew']=[x.replace(" ","") for x in Gvariables['UniqueID']]

In [9]:
Indonesia['UniqueIDNew'].head(20)

0                 SI_POV_DAY1
1                 SI_POV_DAY1
2                 SI_POV_DAY1
3                 SI_POV_DAY1
4                 SI_POV_DAY1
5                 SI_POV_DAY1
6                 SI_POV_DAY1
7                 SI_POV_DAY1
8      SI_POV_EMP1FEMALE15-24
9        SI_POV_EMP1FEMALE15+
10       SI_POV_EMP1FEMALE25+
11       SI_POV_EMP1MALE15-24
12      SI_POV_EMP1BOTHSEX15+
13    SI_POV_EMP1BOTHSEX15-24
14      SI_POV_EMP1BOTHSEX25+
15         SI_POV_EMP1MALE25+
16         SI_POV_EMP1MALE15+
17       SI_POV_EMP1FEMALE15+
18       SI_POV_EMP1FEMALE25+
19      SI_POV_EMP1BOTHSEX15+
Name: UniqueIDNew, dtype: object

In [10]:
Ivariables

,Goal,Target,Indicator,SeriesCode,UniqueID,UniqueIDNew
0,1,1.1,1.1.1,SI_POV_EMP1,SI_POV_EMP1 BOTHSEX 15+,SI_POV_EMP1BOTHSEX15+
1,1,1.2,1.2.1,SI_POV_NAHC,SI_POV_NAHC ALLAREA,SI_POV_NAHCALLAREA
2,1,1.3,1.3.1,SI_COV_SOCAST,SI_COV_SOCAST Q1,SI_COV_SOCASTQ1
3,1,1.4,1.4.1,SP_ACS_BSRVH2O,SP_ACS_BSRVH2O RURAL,SP_ACS_BSRVH2ORURAL
4,1,1.5,1.5.4,SG_DSR_SILN,SG_DSR_SILN,SG_DSR_SILN
...,...,...,...,...,...,...
126,17,17.3,17.3.1,GF_FRN_FDI,GF_FRN_FDI,GF_FRN_FDI
127,17,17.4,17.4.1,DT_TDS_DECT,DT_TDS_DECT,DT_TDS_DECT
128,17,17.6,17.6.1,IT_NET_BBND,IT_NET_BBND ANYS,IT_NET_BBNDANYS
129,17,17.8,17.8.1,IT_USE_ii99,IT_USE_ii99,IT_USE_ii99


## Filter data

Measure the measures picked using Rmd file with the original dataset, to identify which indicator of which target and goal it belongs to, and filters the original dataset for further correlation analysis at target-indicator level.

In [18]:
FilteredIndo=Indonesia.copy()
is_included=FilteredIndo['UniqueIDNew'].isin(Ivariables['UniqueIDNew'])
FilteredIndo=FilteredIndo[is_included]
FilteredIndo.to_csv('/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Indonesia data after selecting one indicator for each target.csv')
FilteredIndo

,UniqueID,Goal,Target,Indicator,SeriesCode,TimePeriod,Value,UniqueIDNew
12,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2012.0,8.30000,SI_POV_EMP1BOTHSEX15+
19,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2013.0,7.00000,SI_POV_EMP1BOTHSEX15+
28,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2014.0,6.10000,SI_POV_EMP1BOTHSEX15+
43,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2015.0,5.70000,SI_POV_EMP1BOTHSEX15+
48,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2016.0,5.20000,SI_POV_EMP1BOTHSEX15+
...,...,...,...,...,...,...,...,...
4225,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2015.0,3409.18378,DC_FTA_TOTAL
4226,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2016.0,1053.97416,DC_FTA_TOTAL
4227,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2017.0,1168.49150,DC_FTA_TOTAL
4228,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2018.0,2042.80551,DC_FTA_TOTAL


In [19]:
FilteredGuate=Guatemala.copy()
is_included=FilteredGuate['UniqueIDNew'].isin(Gvariables['UniqueIDNew'])
FilteredGuate=FilteredGuate[is_included]
FilteredGuate.to_csv('/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Guatemala data after selecting one indicator for each target.csv')
FilteredGuate

,UniqueID,Goal,Target,Indicator,SeriesCode,TimePeriod,Value,UniqueIDNew
4,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2012.0,6.30000,SI_POV_EMP1BOTHSEX15+
10,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2013.0,5.70000,SI_POV_EMP1BOTHSEX15+
25,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2014.0,5.00000,SI_POV_EMP1BOTHSEX15+
28,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2015.0,4.30000,SI_POV_EMP1BOTHSEX15+
38,SI_POV_EMP1 BOTHSEX 15+,1.0,1.1,1.1.1,SI_POV_EMP1,2016.0,4.00000,SI_POV_EMP1BOTHSEX15+
...,...,...,...,...,...,...,...,...
4780,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2015.0,48.66068,DC_FTA_TOTAL
4781,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2016.0,77.71732,DC_FTA_TOTAL
4782,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2017.0,94.32348,DC_FTA_TOTAL
4783,DC_FTA_TOTAL,17.0,17.9,17.9.1,DC_FTA_TOTAL,2018.0,289.73249,DC_FTA_TOTAL


## Preprocess data into desired format

## Calculate correlation among indicators

In [50]:
def calculate_correlation(df):
  Allgoals=df['Goal'].drop_duplicates()
  for goal in Allgoals:
    Alltargets=list(df['Target'].drop_duplicates())
    if len(Alltargets)>1:
      group=df[df['Goal']==goal]
      group=group.drop(columns=['UniqueID','Goal','Indicator','SeriesCode','UniqueIDNew'],axis=1)
      group=group.drop_duplicates(subset = ['Target', 'TimePeriod'])
      group=group.reset_index(drop=True)
      group_pivot=group.pivot(index=['TimePeriod'], columns=['Target'], values='Value').reset_index()
      group_pivot=group_pivot.apply(lambda x:x.astype(float))
      correlation=group_pivot.corr()
      correlation=pd.DataFrame(correlation)
      correlation=correlation.drop('TimePeriod', axis=1)
      correlation=correlation.drop('TimePeriod', axis=0)
      path='/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Indonesia Correlation among Targets/Indonesia Correlation among Targets of Goal '+str(goal)+'.csv'
      correlation.to_csv(path)
      print('Goal '+str(goal)+' completed')

In [51]:
calculate_correlation(FilteredIndo)

Goal 1.0 completed
Goal 2.0 completed
Goal 3.0 completed
Goal 4.0 completed
Goal 5.0 completed
Goal 6.0 completed
Goal 7.0 completed
Goal 8.0 completed
Goal 9.0 completed
Goal 10.0 completed
Goal 11.0 completed
Goal 12.0 completed
Goal 13.0 completed
Goal 14.0 completed
Goal 15.0 completed
Goal 16.0 completed
Goal 17.0 completed


In [52]:
def calculate_correlation(df):
  Allgoals=df['Goal'].drop_duplicates()
  for goal in Allgoals:
    Alltargets=list(df['Target'].drop_duplicates())
    if len(Alltargets)>1:
      group=df[df['Goal']==goal]
      group=group.drop(columns=['UniqueID','Goal','Indicator','SeriesCode','UniqueIDNew'],axis=1)
      group=group.drop_duplicates(subset = ['Target', 'TimePeriod'])
      group=group.reset_index(drop=True)
      group_pivot=group.pivot(index=['TimePeriod'], columns=['Target'], values='Value').reset_index()
      group_pivot=group_pivot.apply(lambda x:x.astype(float))
      correlation=group_pivot.corr()
      correlation=pd.DataFrame(correlation)
      correlation=correlation.drop('TimePeriod', axis=1)
      correlation=correlation.drop('TimePeriod', axis=0)
      path='/content/drive/MyDrive/Colab Notebooks/Practicum SDG Networks/Data/Guatemala Correlation among Targets/Guatemala Correlation among Targets of Goal '+str(goal)+'.csv'
      correlation.to_csv(path)
      print('Goal '+str(goal)+' completed')

In [53]:
calculate_correlation(FilteredGuate)

Goal 1.0 completed
Goal 2.0 completed
Goal 3.0 completed
Goal 4.0 completed
Goal 5.0 completed
Goal 6.0 completed
Goal 7.0 completed
Goal 8.0 completed
Goal 9.0 completed
Goal 10.0 completed
Goal 11.0 completed
Goal 12.0 completed
Goal 13.0 completed
Goal 14.0 completed
Goal 15.0 completed
Goal 16.0 completed
Goal 17.0 completed
